In [ ]:
import logging
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    # Load data from Unity Catalog tables
    customers_df = spark.table("genai_demo.sas.customers")
    transactions_df = spark.table("genai_demo.sas.transactions")

    # Step 1: Filter valid transactions
    valid_txns_df = transactions_df.filter((F.col("Sales") > 0) & (F.col("Product").isNotNull()))

    # Step 2: Calculate Effective Price
    trans_step2_df = valid_txns_df.withColumn("EffectivePrice", F.col("Sales") * (1 - F.col("Discount") / 100))

    # Step 3: Calculate Total Value
    trans_step3_df = trans_step2_df.withColumn("TotalValue", F.col("EffectivePrice") * F.col("Quantity"))

    # Step 4: Join with Customers
    full_data_df = trans_step3_df.join(customers_df, on="CustomerID", how="left")

    # Step 5: Calculate Tenure Days
    trans_step5_df = full_data_df.withColumn("TenureDays", F.datediff(F.col("TransDate"), F.col("JoinDate")))

    # Step 6: Assign Tenure Category
    trans_step6_df = trans_step5_df.withColumn("TenureCategory", 
                                               F.when(F.col("TenureDays") < 180, "New")
                                               .when(F.col("TenureDays") < 365, "Medium")
                                               .otherwise("Loyal"))

    # Step 7: High Value Flag
    trans_step7_df = trans_step6_df.withColumn("HighValueFlag", F.col("TotalValue") > 2000)

    # Step 8: Product Group Classification
    trans_step8_df = trans_step7_df.withColumn("ProductGroup", 
                                               F.when(F.col("Product").isin(["A", "C"]), "Core")
                                               .otherwise("Non-Core"))

    # Step 9: Final Data Preparation
    final_data_df = trans_step8_df

    # Step 10: Sort by ProductGroup for Standardization
    sorted_final_data_df = final_data_df.orderBy("ProductGroup")

    # Step 11: Z-score Standardization
    windowSpec = Window.partitionBy("ProductGroup")
    standardized_df = sorted_final_data_df.withColumn("StandardizedTotalValue", 
                                                      (F.col("TotalValue") - F.avg("TotalValue").over(windowSpec)) / F.stddev("TotalValue").over(windowSpec))

    # Step 12: Outlier Detection
    enhanced_final_data_df = standardized_df.withColumn("OutlierFlag", F.abs(F.col("StandardizedTotalValue")) > 2)

    # Write the final output to Unity Catalog table
    spark.sql("DROP TABLE IF EXISTS genai_demo.sas.enhanced_final_data")
    enhanced_final_data_df.write.format("delta").mode("overwrite").saveAsTable("genai_demo.sas.enhanced_final_data")

    # Generate summary statistics
    summary_stats_df = enhanced_final_data_df.groupBy("Region", "ProductGroup").agg(
        F.mean("TotalValue").alias("MeanTotalValue"),
        F.sum("TotalValue").alias("SumTotalValue"),
        F.mean("Quantity").alias("MeanQuantity"),
        F.sum("Quantity").alias("SumQuantity"),
        F.mean("Sales").alias("MeanSales"),
        F.sum("Sales").alias("SumSales")
    )

    # Log summary statistics
    logger.info("Summary Statistics:")
    summary_stats_df.show()

except Exception as e:
    logger.error(f"An error occurred: {e}")
